# Classification of rotated signals

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import os
import skinematics as skin
import matplotlib.pyplot as plt
import functionsMasterProjectMeinhart as fmpm

## Training Data

In [2]:
# load all data, except data from one subject (test data)
test_data_subject = 1

db_name='DataBase_Physio_with_nonEx.db' # database name
exercise_abbrs = ['RF','RO','RS','LR','BC','TC','MP','SA','P1','P2','NE'] # exercise abbreviations
# Connect to an existing database
conn = sqlite3.connect(db_name)
cur = conn.cursor()
train_data_points = {} # dictionary with the exercise abbreviation as key
for key in exercise_abbrs:
    # sql command to extract data
    query_sql = """
        SELECT r.start_time, r.stop_time, e.csv_file
        FROM subjects s
        INNER JOIN exercises e
        ON s.id = e.subject_id
        INNER JOIN paradigms p
        ON p.id = e.paradigm_id
        INNER JOIN repetitions r
        ON e.id = r.exercise_id
        WHERE p.abbreviation = '{}'
        AND NOT s.id = {}
        """.format(key, test_data_subject)
    # get data from data base and close connection
    train_data_points[key] = pd.read_sql_query(query_sql, conn)
conn.close()

print('Length of the individual data frames:')
num_data_points_train = 0
for key in exercise_abbrs:
    print(key + ':\t' + str(train_data_points[key].shape[0]))
    num_data_points_train += train_data_points[key].shape[0]
print('total:\t' + str(num_data_points_train))

Length of the individual data frames:
RF:	239
RO:	240
RS:	240
LR:	241
BC:	242
TC:	243
MP:	242
SA:	242
P1:	240
P2:	239
NE:	3712
total:	6120


In [3]:
# number of sections to split the signals
number_sections = 10

sampling_rate = 256 # [Hz]
sig_names = ['Acc','Gyr'] # signals which shall be considered for the mean calculation
csv_dir='E:\Physio_Data_Split_Ex_and_NonEx' # directory of csv file
exercise_abbrs = ['RF','RO','RS','LR','BC','TC','MP','SA','P1','P2','NE']
exercise_dict = {ex: ii for ii, ex in enumerate(exercise_abbrs)}

In [4]:
X_train = np.zeros((num_data_points_train, number_sections*6))
y_train = np.zeros(num_data_points_train, dtype=np.int8)

count = 0
y_pos = 0
prev_prog = 0
max_num = num_data_points_train

# go through all exercises
for ex in exercise_abbrs:
    
    # go through all repetitions (data points) of the current exercise
    for ii in range(len(train_data_points[ex])):

        # join file path
        file_path = os.path.join(csv_dir, train_data_points[ex]['csv_file'][ii])

        # load the signal data of the corresponding time range of the current repetition
        selected_data = fmpm.get_sensor_data(in_file = file_path, 
                                             signals = sig_names, 
                                             sampling_rate = sampling_rate, 
                                             start_time = float(train_data_points[ex]['start_time'][ii]), 
                                             stop_time = float(train_data_points[ex]['stop_time'][ii]))
        

        # calculate the corresponding section means of the current repetition
        section_means = fmpm.split_range_into_sections(signal_data = selected_data,
                                                       num_sec = number_sections,
                                                       signals = sig_names)
        
        # generate features
        col = 0
        for sig in sig_names:
            for jj in [0,1,2]: # x, y, z comp. of the corresponding signal
                X_train[count, col:col+number_sections] = section_means[sig][:,jj]
                col += number_sections
        
        count += 1
        prev_prog = fmpm.print_progress(count, max_num, prev_prog)
    
    label = exercise_dict[ex]
    y_train[y_pos:y_pos+len(train_data_points[ex])] = label
    y_pos += len(train_data_points[ex])

Progress: 100%


In [5]:
#plt.plot(X_train[0,:].reshape(6,10).transpose());

## Test Data

In [6]:
# Connect to an existing database
conn = sqlite3.connect(db_name)
cur = conn.cursor()
test_data_points = {} # dictionary with the exercise abbreviation as key
for key in exercise_abbrs:
    # sql command to extract data
    query_sql = """
        SELECT r.start_time, r.stop_time, e.csv_file
        FROM subjects s
        INNER JOIN exercises e
        ON s.id = e.subject_id
        INNER JOIN paradigms p
        ON p.id = e.paradigm_id
        INNER JOIN repetitions r
        ON e.id = r.exercise_id
        WHERE p.abbreviation = '{}'
        AND s.id = {}
        """.format(key, test_data_subject)
    # get data from data base and close connection
    test_data_points[key] = pd.read_sql_query(query_sql, conn)
conn.close()

print('Length of the individual data frames:')
num_data_points_test = 0
for key in exercise_abbrs:
    print(key + ':\t' + str(test_data_points[key].shape[0]))
    num_data_points_test += test_data_points[key].shape[0]
print('total:\t' + str(num_data_points_test))
    

Length of the individual data frames:
RF:	30
RO:	30
RS:	30
LR:	30
BC:	31
TC:	30
MP:	30
SA:	31
P1:	30
P2:	30
NE:	407
total:	709


In [7]:
def rotate_signal(signal_data, axis=0, rot_angle=90, signals=['Acc','Gyr']):
    
    # if no signals are given as keys, select all keys of the input dictionary
    if signals is None:
        signals = [*signal_data]
    
    # create rotation matrix
    R = skin.rotmat.R(axis=axis, angle=rot_angle)
    
    # dictionary for rotated data
    rot_signal_data = {}
    
    # rotate the signals
    for sig in signals: 
        rot_signal_data[sig] = (R @ signal_data[sig].T).T
        
    return rot_signal_data

In [14]:
X_test = np.zeros((num_data_points_test, number_sections*6))
y_test = np.zeros(num_data_points_test, dtype=np.int8)

rot_axis = 0 # 0,1,2 --> x,y,z
rot_angle = 20 # deg

count = 0
y_pos = 0
prev_prog = 0
max_num = num_data_points_test

# go through all exercises
for ex in exercise_abbrs:
    
    # go through all repetitions (data points) of the current exercise
    for ii in range(len(test_data_points[ex])):

        # join file path
        file_path = os.path.join(csv_dir, test_data_points[ex]['csv_file'][ii])

        # load the signal data of the corresponding time range of the current repetition
        selected_data = fmpm.get_sensor_data(in_file = file_path, 
                                             signals = sig_names, 
                                             sampling_rate = sampling_rate, 
                                             start_time = float(test_data_points[ex]['start_time'][ii]), 
                                             stop_time = float(test_data_points[ex]['stop_time'][ii]))
        
        # rotate the signals
        rot_data = rotate_signal(selected_data, axis=rot_axis, rot_angle=rot_angle, signals=['Acc','Gyr'])
        
        # calculate the corresponding section means of the current repetition
        section_means = fmpm.split_range_into_sections(signal_data = rot_data,
                                                       num_sec = number_sections,
                                                       signals = sig_names)
        
        # generate features
        col = 0
        for sig in sig_names:
            for jj in [0,1,2]: # x, y, z comp. of the corresponding signal
                X_test[count, col:col+number_sections] = section_means[sig][:,jj]
                col += number_sections
        
        count += 1
        prev_prog = fmpm.print_progress(count, max_num, prev_prog)
    
    label = exercise_dict[ex]
    y_test[y_pos:y_pos+len(test_data_points[ex])] = label
    y_pos += len(test_data_points[ex])

Progress: 100%


## Classification

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [16]:
# create random forest classifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=40, n_jobs=-1, random_state=42)

# train the model
rnd_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=40,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

### Functions for evaluation

In [17]:
def print_precision_recall_accuracy(y_pred, y_test):
    '''
    This function prints precision, recall and accuracy for each exercise.
    
    Parameters
    ----------
    y_pred : array
        Prediceted classes (0...10).
    
    y_test : array
        Actual classes (0...10).
    
    Returns
    -------
    None
    '''
    
    # exercise abbreviations
    exercise_abbrs = ['RF','RO','RS','LR','BC','TC','MP','SA','P1','P2','NE']
    
    # dictionary for labels
    label_ex = {'RF':0,'RO':1,'RS':2,'LR':3,'BC':4,'TC':5,'MP':6,'SA':7,'P1':8,'P2':9,'NE':10}

    
    print('Exercise\tPrecision [%]\tRecall [%]\tAccuracy[%]')
    
    for ex in exercise_abbrs:
        TP = sum((y_pred == label_ex[ex]) & (np.array(y_test) == label_ex[ex])) # True Positives
        TN = sum((y_pred != label_ex[ex]) & (np.array(y_test) != label_ex[ex])) # True Negatives
        FP = sum((y_pred == label_ex[ex]) & (np.array(y_test) != label_ex[ex])) # False Positives
        FN = sum((y_pred != label_ex[ex]) & (np.array(y_test) == label_ex[ex])) # False Negatives

        precision = TP / (TP+FP)
        recall = TP / (TP+FN)
        accuracy = (TP+TN) / (TP+TN+FP+FN)
        
        print('  '+ ex +'\t\t  {:6.2f}'.format(precision*100)+ \
              '\t  {:6.2f}'.format(recall*100)+'\t  {:6.2f}'.format(accuracy*100))

In [18]:
def print_misclassified_data_points(y_pred, y_test):
    '''
    This function prints all misclassified data points.
    
    Parameters
    ----------
    y_pred : array
        Predicted classes (0...10).
    
    y_test : array
        Actual classes (0...10).
    
    Returns
    -------
    None
    '''
    
    # exercise abbreviations
    exercise_abbrs = ['RF','RO','RS','LR','BC','TC','MP','SA','P1','P2','NE'] 
    
    # indices of misclassified data points
    ind_misclassified = np.flatnonzero(y_test != y_pred) 

    # print misclassified data points
    print('{0} misclassified ({1} test data points):'.format(sum(y_test != y_pred), len(y_test)))
    for ii in ind_misclassified:
        print(exercise_abbrs[y_test[ii]] + ' classified as ' + exercise_abbrs[y_pred[ii]])

In [19]:
# predict labels
y_pred = rnd_clf.predict(X_test)

# show results
print('Model: Random Forest Classifier\n')
print('Features: Means of {} sections per signal (3 x Acc + 3 x Gyr) --> {} features\n'.format(number_sections,
                                                                                              number_sections*6))
print('Total Accuracy: {:.5f}'.format((accuracy_score(y_test, y_pred))))
print('\n')
print_precision_recall_accuracy(y_pred, y_test)
print('\n')
print_misclassified_data_points(y_pred, y_test)

Model: Random Forest Classifier

Features: Means of 10 sections per signal (3 x Acc + 3 x Gyr) --> 60 features

Total Accuracy: 0.97884


Exercise	Precision [%]	Recall [%]	Accuracy[%]
  RF		   83.33	  100.00	   99.15
  RO		  100.00	   80.00	   99.15
  RS		  100.00	  100.00	  100.00
  LR		  100.00	   93.33	   99.72
  BC		  100.00	  100.00	  100.00
  TC		  100.00	  100.00	  100.00
  MP		   81.08	  100.00	   99.01
  SA		  100.00	  100.00	  100.00
  P1		  100.00	  100.00	  100.00
  P2		  100.00	  100.00	  100.00
  NE		   99.50	   98.28	   98.73


15 misclassified (709 test data points):
RO classified as RF
RO classified as RF
RO classified as RF
RO classified as RF
RO classified as RF
RO classified as RF
LR classified as NE
LR classified as NE
NE classified as MP
NE classified as MP
NE classified as MP
NE classified as MP
NE classified as MP
NE classified as MP
NE classified as MP
